# INT8 Quantization with Post-training Optimization Tool (POT) in Simplified Mode tutorial

This tutorial shows how to quantize a [ResNet20](https://github.com/chenyaofo/pytorch-cifar-models) image classification model, trained on [CIFAR10 ](http://pytorch.org/vision/main/generated/torchvision.datasets.CIFAR10.html) dataset, using the Simplified Mode of OpenVINO Post-Training Optimization Tool (POT).

Simplified Mode is designed to make the data preparation step easier, before model optimization. The mode is represented by an implementation of the engine interface in the POT API. It enables reading data from an arbitrary folder specified by the user. Currently, Simplified Mode is available only for image data in PNG or JPEG formats, stored in a single folder.

**Note:** This mode cannot be used with the accuracy-aware method. It is not possible to control accuracy after optimization using this mode. However, Simplified Mode can be useful for estimating performance improvements when optimizing models.

This tutorial includes the following steps:

- Downloading and saving the CIFAR10 dataset
- Preparing the model for quantization
- Compressing the prepared model
- Measuring and comparing the performance of the original and quantized models
- Demonstrating the use of the quantized model for image classification


In [ ]:
import os
from pathlib import Path
import warnings

import torch
from torchvision import transforms as T
from torchvision.datasets import CIFAR10

import matplotlib.pyplot as plt
import numpy as np

from openvino.runtime import Core, Tensor

warnings.filterwarnings("ignore")

# Set the data and model directories
MODEL_DIR = 'model'
CALIB_DIR = 'calib'
CIFAR_DIR = 'cifar'
CALIB_SET_SIZE = 300
MODEL_NAME = 'resnet20'

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CALIB_DIR, exist_ok=True)

## Prepare the calibration dataset
The following steps are required to prepare the calibration dataset:
- Download CIFAR10 dataset from Torchvision.datasets repository
- Save the selected number of elements from this dataset as .png images in a separate folder

In [ ]:
transform = T.Compose([T.ToTensor()])
dataset = CIFAR10(root=CIFAR_DIR, train=False, transform=transform, download=True)

In [ ]:
pil_converter = T.ToPILImage(mode="RGB")

for idx, info in enumerate(dataset):
    im = info[0]
    if idx >= CALIB_SET_SIZE:
        break
    label = info[1]
    pil_converter(im.squeeze(0)).save(Path(CALIB_DIR) / f'{label}_{idx}.png')

## Prepare the Model
Model preparation includes the following steps:,
- Download PyTorch model from Torchvision repository,
- Convert the model to ONNX format,
- Run OpenVINO Model Optimizer tool to convert ONNX to OpenVINO Intermediate Representation (IR)

In [ ]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
dummy_input = torch.randn(1, 3, 32, 32)

onnx_model_path = Path(MODEL_DIR) / '{}.onnx'.format(MODEL_NAME)
ir_model_xml = onnx_model_path.with_suffix('.xml')
ir_model_bin = onnx_model_path.with_suffix('.bin')

torch.onnx.export(model, dummy_input, onnx_model_path)

Now we convert this model into the OpenVINO IR using the Model Optimizer:

In [ ]:
!mo --framework=onnx --data_type=FP32 --input_shape=[1,3,32,32] -m $onnx_model_path  --output_dir $MODEL_DIR

## Compression stage
Compress the model with the following command:
  
`pot -q default -m <path_to_xml> -w <path_to_bin> --engine simplified --data-source <path_to_data>`

In [ ]:
!pot -q default -m $ir_model_xml -w $ir_model_bin --engine simplified --data-source $CALIB_DIR --output-dir compressed --direct-dump --name $MODEL_NAME

## Compare Performance of the Original and Quantized Models

Finally, we will measure the inference performance of the FP32 and INT8 models. To do this, we use [Benchmark Tool](https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html) - an inference performance measurement tool for OpenVINO.

**NOTE:** For more accurate performance, we recommended running benchmark_app in a terminal/command prompt after closing other applications. Run benchmark_app -m model.xml -d CPU to benchmark async inference on CPU for one minute. Change CPU to GPU to benchmark on GPU. Run benchmark_app --help to see an overview of all command line options.

In [ ]:
optimized_model_path = Path('compressed/optimized')
optimized_model_xml = optimized_model_path / '{}.xml'.format(MODEL_NAME)
optimized_model_bin = optimized_model_path / '{}.bin'.format(MODEL_NAME)

In [ ]:
# Inference FP32 model (IR)
!benchmark_app -m $ir_model_xml -d CPU -api async

In [ ]:
# Inference INT8 model (IR)
!benchmark_app -m $optimized_model_xml -d CPU -api async

## Demonstration of the results

This section demonstrates how to use the compressed model by running the optimized model on a subset of images from the CIFAR10 dataset and shows predictions using the model.

The first step is to load the model:

In [ ]:
ie = Core()

compiled_model = ie.compile_model(str(optimized_model_xml), "AUTO")

In [ ]:
# define all possible labels from CIFAR10
labels_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
all_images = []
all_labels = []

# get all images and their labels 
for batch in dataset:
    all_images.append(torch.unsqueeze(batch[0], 0))
    all_labels.append(batch[1])

This section defines the function that shows the images and their labels using the indexes and two lists created in the previous step:

In [ ]:
def plot_pictures(indexes: list, images=all_images, labels=all_labels):
    """Plot images with the specified indexes.
    :param indexes: a list of indexes of images to be displayed.
    :param images: a list of images from the dataset.
    :param labels: a list of labels for each image.
    """
    num_pics = len(indexes)
    _, axarr = plt.subplots(1, num_pics)
    for idx, im_idx in enumerate(indexes):
        assert idx < 10000, 'Cannot get such index, there are only 10000'
        pic = np.rollaxis(images[im_idx].squeeze().numpy(), 0, 3)
        axarr[idx].imshow(pic)
        axarr[idx].set_title(labels_names[labels[im_idx]])

In this section we define a function that uses the optimized model to obtain predictions for the selected images:

In [ ]:
def infer_on_images(net, indexes: list, images=all_images):
    """ Inference model on a set of images.
    :param net: model on which do inference
    :param indexes: a list of indexes of images to infer on.
    :param images: a list of images from the dataset.
    """
    predicted_labels = []
    infer_request = net.create_infer_request()
    for idx in indexes:
        assert idx < 10000, 'Cannot get such index, there are only 10000'
        input_tensor = Tensor(array=images[idx].detach().numpy(), shared_memory=True)
        infer_request.set_input_tensor(input_tensor)
        infer_request.start_async()
        infer_request.wait()
        output = infer_request.get_output_tensor()
        result = list(output.data)
        result = labels_names[np.argmax(result[0])]
        predicted_labels.append(result)
    return predicted_labels

In [ ]:
indexes_to_infer = [0, 1, 2]  # to plot specify indexes

plot_pictures(indexes_to_infer)

results_quanized = infer_on_images(compiled_model, indexes_to_infer)

print(f"Image labels using the quantized model : {results_quanized}.")